# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
from pickle import load
import pickle
import DiadFit as pf
pf.__version__

'0.0.59'

In [3]:
MasterFolder = r"C:\Users\charl\Documents\Instruments and calibrations"
DayFolder = os.path.join(MasterFolder, "UCB_4-10-23")

meta_path=DayFolder +  '\Metadata'
spectra_path=DayFolder + '\Spectra'

if not os.path.exists(MasterFolder + '\OUTPUT'):
    os.mkdir(MasterFolder +'\OUTPUT')

output_path=MasterFolder +'\OUTPUT'


## Load in specra, and combine

In [4]:
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [5]:
df2.head()

,Unnamed: 0,filename,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,103 FDCA-P25-1,104.586050,0.003912,0.003912,1282.436956,0.000000,749.381956,1282.436956,2128.474296,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,105 FDCA-P26-1,104.545273,0.010529,0.007639,1282.504156,0.006473,367.496596,1282.504156,1052.928202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,187 FDCA-P59-1,103.586720,0.008811,0.006356,1284.643370,0.005293,584.887389,1284.643370,1239.700037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,199 FDCA-P62-3,103.478532,0.008305,0.006030,1284.874481,0.005119,974.245489,1284.874481,1826.824194,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,201 FDCA-P63-1_CRR_DiadFit,103.451626,0.007969,0.005669,1284.949618,0.004427,990.414901,1284.949618,1792.442986,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [6]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'Si', 'series','IMG','Cap','window','nodiad'],
 sort=True)
diad_meta

['003 FDCA-P1-1.txt',
 '005 FDCA-P1-2.txt',
 '007 FDCA-P1-3.txt',
 '009 FDCA-P2-1.txt',
 '011 FDCA-P2-2.txt',
 '013 FDCA-P2-3.txt',
 '015 FDCA-P3-1.txt',
 '017 FDCA-P3-2.txt',
 '019 FDCA-P3-3.txt',
 '022 FDCA-P4-1.txt',
 '024 FDCA-P4-2.txt',
 '026 FDCA-P4-3.txt',
 '028 FDCA-P5-1.txt',
 '030 FDCA-P5-2.txt',
 '032 FDCA-P5-3.txt',
 '034 FDCA-P6-1.txt',
 '036 FDCA-P6-2.txt',
 '038 FDCA-P6-3.txt',
 '040 FDCA-P7-1.txt',
 '042 FDCA-P7-2.txt',
 '044 FDCA-P7-3.txt',
 '049 FDCA-P8-1.txt',
 '051 FDCA-P8-2.txt',
 '053 FDCA-P8-3.txt',
 '055 FDCA-P9-1.txt',
 '057 FDCA-P9-2.txt',
 '059 FDCA-P9-3.txt',
 '061 FDCA-P10-1.txt',
 '063 FDCA-P10-2.txt',
 '065 FDCA-P10-3.txt',
 '067 FDCA-P11-1.txt',
 '069 FDCA-P12-1.txt',
 '071 FDCA-P13-1.txt',
 '073 FDCA-P14-1.txt',
 '075 FDCA-P14-2.txt',
 '076 FDCA-P14-3.txt',
 '078 FDCA-P15-1.txt',
 '080 FDCA-P16-1.txt',
 '082 FDCA-P17-1.txt',
 '084 FDCA-P18-1.txt',
 '085 FDCA-P18-2.txt',
 '087 FDCA-P19-1.txt',
 '088 FDCA-P19-2.txt',
 '089 FDCA-P19-3.txt',
 '091 FDCA-P20-

## Now get all the important information out of the metadata files

In [7]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=True)
meta['filename'].iloc[0]

100%|██████████| 156/156 [00:00<00:00, 598.26it/s]

Done
Oof, try not to work after midnight!


'156 FDCA-P45-1.txt'

## Then get a simple file name you can stitch with the spectra

In [8]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'156 FDCA-P45-1'

## Now get filename from spectra into same form as metadata

In [9]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

good job, no duplicate file names


In [10]:
file_s

array(['103 FDCA-P25-1', '105 FDCA-P26-1', '187 FDCA-P59-1',
       '199 FDCA-P62-3', '201 FDCA-P63-1', '202 FDCA-P63-2',
       '203 FDCA-P63-3', '205 FDCA-P64-1', '206 FDCA-P64-2',
       '207 FDCA-P64-3', '209 FDCA-P65-1', '210 FDCA-P65-2',
       '211 FDCA-P65-3', '215 FDCA-P66-3', '217 FDCA-P67-1',
       '218 FDCA-P67-2', '219 FDCA-P67-3', '221 FDCA-P68-1',
       '222 FDCA-P68-2', '223 FDCA-P68-3', '225 FDCA-P69-1',
       '226 FDCA-P69-2', '227 FDCA-P69-3', '229 FDCA-P70-1',
       '230 FDCA-P70-2', '231 FDCA-P70-3', '233 FDCA-P71-1',
       '234 FDCA-P71-2', '237 FDCA-P72-1', '238 FDCA-P72-2',
       '239 FDCA-P72-3', '241 FDCA-P73-1', '242 FDCA-P73-2',
       '243 FDCA-P73-3', '245 FDCA-P74-1', '246 FDCA-P74-2',
       '247 FDCA-P74-3', '249 FDCA-P75-1', '250 FDCA-P75-2',
       '251 FDCA-P75-3', '253 FDCA-P76-1', '254 FDCA-P76-2',
       '255 FDCA-P76-3', '003 FDCA-P1-1', '005 FDCA-P1-2',
       '007 FDCA-P1-3', '009 FDCA-P2-1', '011 FDCA-P2-2', '013 FDCA-P2-3',
       '015 

In [11]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [12]:
Ne_corr=pf.calculate_Ne_corr_std_err_values(pickle_str='polyfit_data.pkl', 
    new_x=df_combo['sec since midnight'], CI=0.67)
Ne_corr.head()


3


,time,preferred_values,lower_values,upper_values
0,72526.0,0.997552,0.997543,0.997560
1,72874.0,0.997552,0.997544,0.997561
2,94759.0,0.997576,0.997568,0.997585
3,97637.0,0.997576,0.997567,0.997584
4,98048.0,0.997576,0.997567,0.997584


In [13]:

split_err=pf.propagate_errors_for_splitting(Ne_corr, df_combo)
df_combo_out=df_combo.copy()
df_combo_out.insert(1, 'Corrected_Splitting', df_combo['Splitting']*Ne_corr['preferred_values'])
df_combo_out.insert(2, 'Corr_Split+1σ', df_combo_out['Corrected_Splitting']+split_err)
df_combo_out.insert(3, 'Corr_Split-1σ',df_combo_out['Corrected_Splitting']-split_err)
df_combo_out.insert(4, 'Corr_Split_1σ_val', split_err)

nm=os.path.basename(DayFolder)
df_combo_out.to_excel(output_path+'/'+nm+'_FI_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [17]:
if path.exists('Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel('Carb_Peak_fits.xlsx')
else:
    Carb=None
if path.exists('SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel('SO2_Peak_fits.xlsx')
else:
    SO2=None
if SO2 is not None and Carb is not None:
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
elif SO2 is not None and Carb is None:
    Sec_Phases=SO2
elif SO2 is None and Carb is not None:
    Sec_Phases=Carb
else:
    Sec_Phases=None
Sec_Phases.head()

,filename,Peak_Cent_Carb,Peak_Area_Carb,Peak_Height_Carb,Model_name


In [18]:
# Remove these to get the pure file name
if Sec_Phases is not None:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')

    file_sec_phase

good job, no duplicate file names


In [19]:
file_sec_phase

array([], dtype=object)

In [20]:
df_combo['name_for_matching']

0      103 FDCA-P25-1
1      105 FDCA-P26-1
2      187 FDCA-P59-1
3      199 FDCA-P62-3
4      201 FDCA-P63-1
            ...      
151    197 FDCA-P62-1
152    198 FDCA-P62-2
153    213 FDCA-P66-1
154    214 FDCA-P66-2
155    235 FDCA-P71-3
Name: name_for_matching, Length: 156, dtype: object

In [21]:
df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']

if Sec_Phases is not None:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo_sec_phase=df_combo_out.merge(Sec_Phases, 
    on='name_for_matching', how='outer')
    
else:
    df_combo_sec_phase=df_combo_out

### Add some more useful columns

In [22]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
    
df_combo_sec_phase.head()

,Unnamed: 0,Corrected_Splitting,Corr_Split+1σ,Corr_Split-1σ,Corr_Split_1σ_val,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,...,duration,24hr_time,sec since midnight,Spectral Center,filename,Peak_Cent_Carb,Peak_Area_Carb,Peak_Height_Carb,Model_name,Carb_Diad_Ratio
0,0,104.329993,104.333997,104.325988,0.004004,103 FDCA-P25-1,104.586050,0.003912,0.003912,1282.436956,...,"['0h', '1m', '42s']",8:08:46 PM\n,72526.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN
1,0,104.289394,104.297067,104.281721,0.007673,105 FDCA-P26-1,104.545273,0.010529,0.007639,1282.504156,...,"['0h', '1m', '42s']",8:14:34 PM\n,72874.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN
2,0,103.335668,103.342071,103.329265,0.006403,187 FDCA-P59-1,103.586720,0.008811,0.006356,1284.643370,...,"['0h', '1m', '41s']",2:19:19 AM\n,94759.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN
3,0,103.227680,103.233761,103.221599,0.006081,199 FDCA-P62-3,103.478532,0.008305,0.006030,1284.874481,...,"['0h', '2m', '31s']",3:07:17 AM\n,97637.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN
4,0,103.200825,103.206551,103.195099,0.005726,201 FDCA-P63-1_CRR_DiadFit,103.451626,0.007969,0.005669,1284.949618,...,"['0h', '2m', '16s']",3:14:08 AM\n,98048.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_combo_sec_phase.to_excel(output_path+'/'+nm+'_FI_fitting_w_sec_phases.xlsx')


In [19]:
from winotify import Notification, audio

toast= Notification(app_id="VSCode",title="Notebook completed",
                    msg="Step4_Stitch_Outputs_Together is done!",
                    duration="short")
toast.set_audio(audio.Mail,loop=False)
toast.show()